In [40]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, AveragePooling2D
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import tensorflow_datasets as tfds  # pip install tensorflow-datasets
import tensorflow as tf
import logging
import numpy as np
import time

In [41]:
def cifar10_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), padding='same',
    activation='relu', input_shape=(32, 32, 3)))
  model.add(Conv2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(Conv2D(64, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  model.compile(loss='categorical_crossentropy',
    optimizer=SGD(lr=.1, momentum=0.9, nesterov=True),
    metrics=['accuracy'])
  return model


In [42]:
def cifar10_train(model):
  (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
  y_train = keras.utils.to_categorical(y_train, 10)
  y_test = keras.utils.to_categorical(y_test, 10)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255.0
  x_test /= 255.0

  def scheduler(epoch):
    if epoch < 30:
      return 0.01
    if epoch < 50:
      return 0.001
    return 0.0001

  change_lr = keras.callbacks.LearningRateScheduler(scheduler)

  t_start = time.time()

  # randomly rotate images in the range (degrees, 0 to 180)
  datagen = keras.preprocessing.image.ImageDataGenerator(rotation_range=0,
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    # randomly flip images
    horizontal_flip=True,
    # randomly flip images
    vertical_flip=False)

  # Compute quantities required for feature-wise normalization
  # (std, mean, and principal components if ZCA whitening is applied).
  datagen.fit(x_train)

  # Fit the model on the batches generated by datagen.flow().
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=128),
    epochs=60, callbacks=[change_lr],
    validation_data=(x_test, y_test))
  print("Training done, dT:", time.time() - t_start)

In [43]:
model = cifar10_model()
cifar10_train(model)
model.save('cifar10_32x32.h5')

Epoch 1/60


204/391 [==============>...............] - ETA: 16:36 - loss: 2.3030 - acc: 0.10 - ETA: 10:12 - loss: 2.3136 - acc: 0.08 - ETA: 7:58 - loss: 2.3057 - acc: 0.0833 - ETA: 6:56 - loss: 2.3051 - acc: 0.089 - ETA: 6:17 - loss: 2.3056 - acc: 0.084 - ETA: 5:52 - loss: 2.3070 - acc: 0.084 - ETA: 5:32 - loss: 2.3073 - acc: 0.088 - ETA: 5:17 - loss: 2.3064 - acc: 0.092 - ETA: 5:06 - loss: 2.3044 - acc: 0.096 - ETA: 4:56 - loss: 2.3048 - acc: 0.096 - ETA: 4:49 - loss: 2.3028 - acc: 0.101 - ETA: 4:43 - loss: 2.3027 - acc: 0.100 - ETA: 4:38 - loss: 2.3025 - acc: 0.099 - ETA: 4:32 - loss: 2.3012 - acc: 0.100 - ETA: 4:28 - loss: 2.3020 - acc: 0.100 - ETA: 4:25 - loss: 2.3015 - acc: 0.103 - ETA: 4:22 - loss: 2.3007 - acc: 0.102 - ETA: 4:20 - loss: 2.3003 - acc: 0.102 - ETA: 4:17 - loss: 2.2997 - acc: 0.104 - ETA: 4:15 - loss: 2.2995 - acc: 0.102 - ETA: 4:12 - loss: 2.2995 - acc: 0.101 - ETA: 4:10 - loss: 2.2991 - acc: 0.103 - ETA: 4:08 - loss: 2.2984 - acc: 0.102 - ETA: 4:05 - loss: 2.2975 - acc: 0.10

391/391 [==============================] - ETA: 1:17 - loss: 2.1206 - acc: 0.207 - ETA: 1:17 - loss: 2.1201 - acc: 0.207 - ETA: 1:16 - loss: 2.1193 - acc: 0.207 - ETA: 1:16 - loss: 2.1182 - acc: 0.208 - ETA: 1:15 - loss: 2.1187 - acc: 0.208 - ETA: 1:15 - loss: 2.1179 - acc: 0.208 - ETA: 1:14 - loss: 2.1176 - acc: 0.208 - ETA: 1:14 - loss: 2.1169 - acc: 0.209 - ETA: 1:13 - loss: 2.1164 - acc: 0.209 - ETA: 1:12 - loss: 2.1159 - acc: 0.209 - ETA: 1:12 - loss: 2.1152 - acc: 0.209 - ETA: 1:11 - loss: 2.1146 - acc: 0.210 - ETA: 1:11 - loss: 2.1137 - acc: 0.210 - ETA: 1:10 - loss: 2.1129 - acc: 0.210 - ETA: 1:10 - loss: 2.1122 - acc: 0.211 - ETA: 1:09 - loss: 2.1114 - acc: 0.211 - ETA: 1:09 - loss: 2.1104 - acc: 0.212 - ETA: 1:08 - loss: 2.1100 - acc: 0.212 - ETA: 1:08 - loss: 2.1092 - acc: 0.213 - ETA: 1:07 - loss: 2.1084 - acc: 0.213 - ETA: 1:07 - loss: 2.1081 - acc: 0.214 - ETA: 1:06 - loss: 2.1072 - acc: 0.214 - ETA: 1:06 - loss: 2.1066 - acc: 0.214 - ETA: 1:05 - loss: 2.1058 - acc: 0.215

207/391 [==============>...............] - ETA: 1:32 - loss: 1.7838 - acc: 0.343 - ETA: 1:32 - loss: 1.7339 - acc: 0.378 - ETA: 1:33 - loss: 1.7783 - acc: 0.349 - ETA: 1:34 - loss: 1.7650 - acc: 0.357 - ETA: 1:34 - loss: 1.7613 - acc: 0.362 - ETA: 1:33 - loss: 1.7579 - acc: 0.369 - ETA: 1:33 - loss: 1.7698 - acc: 0.371 - ETA: 1:33 - loss: 1.7848 - acc: 0.363 - ETA: 1:32 - loss: 1.7800 - acc: 0.362 - ETA: 1:32 - loss: 1.7826 - acc: 0.362 - ETA: 1:31 - loss: 1.7831 - acc: 0.359 - ETA: 1:31 - loss: 1.7756 - acc: 0.358 - ETA: 1:31 - loss: 1.7716 - acc: 0.358 - ETA: 1:31 - loss: 1.7678 - acc: 0.361 - ETA: 1:31 - loss: 1.7648 - acc: 0.360 - ETA: 1:30 - loss: 1.7584 - acc: 0.358 - ETA: 1:30 - loss: 1.7579 - acc: 0.357 - ETA: 1:30 - loss: 1.7522 - acc: 0.358 - ETA: 1:30 - loss: 1.7495 - acc: 0.358 - ETA: 1:30 - loss: 1.7532 - acc: 0.356 - ETA: 1:29 - loss: 1.7537 - acc: 0.355 - ETA: 1:30 - loss: 1.7483 - acc: 0.355 - ETA: 1:29 - loss: 1.7527 - acc: 0.354 - ETA: 1:29 - loss: 1.7510 - acc: 0.353

391/391 [==============================] - ETA: 47s - loss: 1.6983 - acc: 0.37 - ETA: 46s - loss: 1.6982 - acc: 0.37 - ETA: 46s - loss: 1.6981 - acc: 0.37 - ETA: 46s - loss: 1.6975 - acc: 0.37 - ETA: 46s - loss: 1.6974 - acc: 0.37 - ETA: 45s - loss: 1.6970 - acc: 0.37 - ETA: 45s - loss: 1.6967 - acc: 0.37 - ETA: 45s - loss: 1.6969 - acc: 0.37 - ETA: 45s - loss: 1.6968 - acc: 0.37 - ETA: 45s - loss: 1.6961 - acc: 0.37 - ETA: 44s - loss: 1.6960 - acc: 0.37 - ETA: 44s - loss: 1.6956 - acc: 0.37 - ETA: 44s - loss: 1.6950 - acc: 0.37 - ETA: 44s - loss: 1.6945 - acc: 0.37 - ETA: 43s - loss: 1.6943 - acc: 0.37 - ETA: 43s - loss: 1.6940 - acc: 0.37 - ETA: 43s - loss: 1.6933 - acc: 0.37 - ETA: 42s - loss: 1.6928 - acc: 0.37 - ETA: 42s - loss: 1.6919 - acc: 0.37 - ETA: 42s - loss: 1.6910 - acc: 0.37 - ETA: 42s - loss: 1.6902 - acc: 0.37 - ETA: 41s - loss: 1.6902 - acc: 0.37 - ETA: 41s - loss: 1.6894 - acc: 0.37 - ETA: 41s - loss: 1.6895 - acc: 0.37 - ETA: 41s - loss: 1.6892 - acc: 0.37 - ETA: 40

...

391/391 [==============================] - ETA: 44s - loss: 0.5840 - acc: 0.79 - ETA: 44s - loss: 0.5844 - acc: 0.79 - ETA: 44s - loss: 0.5849 - acc: 0.79 - ETA: 44s - loss: 0.5844 - acc: 0.79 - ETA: 43s - loss: 0.5847 - acc: 0.79 - ETA: 43s - loss: 0.5844 - acc: 0.79 - ETA: 43s - loss: 0.5848 - acc: 0.79 - ETA: 43s - loss: 0.5846 - acc: 0.79 - ETA: 42s - loss: 0.5855 - acc: 0.79 - ETA: 42s - loss: 0.5850 - acc: 0.79 - ETA: 42s - loss: 0.5844 - acc: 0.79 - ETA: 42s - loss: 0.5839 - acc: 0.79 - ETA: 41s - loss: 0.5845 - acc: 0.79 - ETA: 41s - loss: 0.5845 - acc: 0.79 - ETA: 41s - loss: 0.5848 - acc: 0.79 - ETA: 41s - loss: 0.5853 - acc: 0.79 - ETA: 41s - loss: 0.5855 - acc: 0.79 - ETA: 40s - loss: 0.5855 - acc: 0.79 - ETA: 40s - loss: 0.5854 - acc: 0.79 - ETA: 40s - loss: 0.5852 - acc: 0.79 - ETA: 40s - loss: 0.5855 - acc: 0.79 - ETA: 39s - loss: 0.5861 - acc: 0.79 - ETA: 39s - loss: 0.5867 - acc: 0.79 - ETA: 39s - loss: 0.5865 - acc: 0.79 - ETA: 39s - loss: 0.5866 - acc: 0.79 - ETA: 38

206/391 [==============>...............] - ETA: 1:47 - loss: 0.6811 - acc: 0.757 - ETA: 1:45 - loss: 0.6074 - acc: 0.777 - ETA: 1:45 - loss: 0.6075 - acc: 0.768 - ETA: 1:46 - loss: 0.6089 - acc: 0.779 - ETA: 1:47 - loss: 0.6109 - acc: 0.787 - ETA: 1:46 - loss: 0.5903 - acc: 0.789 - ETA: 1:45 - loss: 0.5740 - acc: 0.795 - ETA: 1:45 - loss: 0.5763 - acc: 0.799 - ETA: 1:46 - loss: 0.5764 - acc: 0.803 - ETA: 1:45 - loss: 0.5713 - acc: 0.807 - ETA: 1:45 - loss: 0.5726 - acc: 0.806 - ETA: 1:45 - loss: 0.5796 - acc: 0.803 - ETA: 1:45 - loss: 0.5818 - acc: 0.798 - ETA: 1:44 - loss: 0.5786 - acc: 0.798 - ETA: 1:44 - loss: 0.5726 - acc: 0.801 - ETA: 1:44 - loss: 0.5767 - acc: 0.799 - ETA: 1:44 - loss: 0.5704 - acc: 0.802 - ETA: 1:43 - loss: 0.5674 - acc: 0.803 - ETA: 1:43 - loss: 0.5648 - acc: 0.804 - ETA: 1:43 - loss: 0.5675 - acc: 0.803 - ETA: 1:43 - loss: 0.5663 - acc: 0.805 - ETA: 1:42 - loss: 0.5687 - acc: 0.802 - ETA: 1:42 - loss: 0.5680 - acc: 0.803 - ETA: 1:42 - loss: 0.5668 - acc: 0.802

391/391 [==============================] - ETA: 51s - loss: 0.5700 - acc: 0.79 - ETA: 51s - loss: 0.5695 - acc: 0.80 - ETA: 50s - loss: 0.5700 - acc: 0.80 - ETA: 50s - loss: 0.5706 - acc: 0.80 - ETA: 50s - loss: 0.5707 - acc: 0.79 - ETA: 50s - loss: 0.5703 - acc: 0.80 - ETA: 49s - loss: 0.5701 - acc: 0.80 - ETA: 49s - loss: 0.5711 - acc: 0.80 - ETA: 49s - loss: 0.5711 - acc: 0.80 - ETA: 48s - loss: 0.5710 - acc: 0.80 - ETA: 48s - loss: 0.5715 - acc: 0.79 - ETA: 48s - loss: 0.5718 - acc: 0.79 - ETA: 48s - loss: 0.5721 - acc: 0.79 - ETA: 47s - loss: 0.5716 - acc: 0.79 - ETA: 47s - loss: 0.5721 - acc: 0.79 - ETA: 47s - loss: 0.5718 - acc: 0.79 - ETA: 46s - loss: 0.5719 - acc: 0.80 - ETA: 46s - loss: 0.5715 - acc: 0.80 - ETA: 46s - loss: 0.5713 - acc: 0.80 - ETA: 46s - loss: 0.5713 - acc: 0.80 - ETA: 45s - loss: 0.5710 - acc: 0.80 - ETA: 45s - loss: 0.5710 - acc: 0.80 - ETA: 45s - loss: 0.5711 - acc: 0.80 - ETA: 44s - loss: 0.5713 - acc: 0.79 - ETA: 44s - loss: 0.5714 - acc: 0.79 - ETA: 44

207/391 [==============>...............] - ETA: 1:38 - loss: 0.6236 - acc: 0.742 - ETA: 1:42 - loss: 0.6389 - acc: 0.742 - ETA: 1:40 - loss: 0.6217 - acc: 0.770 - ETA: 1:40 - loss: 0.6213 - acc: 0.765 - ETA: 1:40 - loss: 0.6204 - acc: 0.767 - ETA: 1:38 - loss: 0.6026 - acc: 0.779 - ETA: 1:38 - loss: 0.5973 - acc: 0.783 - ETA: 1:38 - loss: 0.5875 - acc: 0.789 - ETA: 1:38 - loss: 0.5883 - acc: 0.790 - ETA: 1:39 - loss: 0.5781 - acc: 0.796 - ETA: 1:39 - loss: 0.5974 - acc: 0.794 - ETA: 1:38 - loss: 0.5987 - acc: 0.792 - ETA: 1:37 - loss: 0.5974 - acc: 0.792 - ETA: 1:37 - loss: 0.5960 - acc: 0.791 - ETA: 1:38 - loss: 0.5949 - acc: 0.791 - ETA: 1:37 - loss: 0.5881 - acc: 0.794 - ETA: 1:38 - loss: 0.5905 - acc: 0.794 - ETA: 1:38 - loss: 0.6000 - acc: 0.793 - ETA: 1:37 - loss: 0.6062 - acc: 0.791 - ETA: 1:37 - loss: 0.6034 - acc: 0.793 - ETA: 1:36 - loss: 0.6015 - acc: 0.794 - ETA: 1:36 - loss: 0.6067 - acc: 0.793 - ETA: 1:36 - loss: 0.6036 - acc: 0.792 - ETA: 1:35 - loss: 0.6083 - acc: 0.791

391/391 [==============================] - ETA: 45s - loss: 0.5802 - acc: 0.79 - ETA: 45s - loss: 0.5799 - acc: 0.79 - ETA: 45s - loss: 0.5799 - acc: 0.79 - ETA: 45s - loss: 0.5793 - acc: 0.79 - ETA: 44s - loss: 0.5791 - acc: 0.79 - ETA: 44s - loss: 0.5784 - acc: 0.79 - ETA: 44s - loss: 0.5783 - acc: 0.79 - ETA: 44s - loss: 0.5780 - acc: 0.79 - ETA: 43s - loss: 0.5781 - acc: 0.79 - ETA: 43s - loss: 0.5782 - acc: 0.79 - ETA: 43s - loss: 0.5773 - acc: 0.79 - ETA: 43s - loss: 0.5775 - acc: 0.79 - ETA: 42s - loss: 0.5777 - acc: 0.79 - ETA: 42s - loss: 0.5773 - acc: 0.79 - ETA: 42s - loss: 0.5774 - acc: 0.79 - ETA: 42s - loss: 0.5769 - acc: 0.79 - ETA: 41s - loss: 0.5771 - acc: 0.79 - ETA: 41s - loss: 0.5772 - acc: 0.79 - ETA: 41s - loss: 0.5775 - acc: 0.80 - ETA: 41s - loss: 0.5768 - acc: 0.80 - ETA: 40s - loss: 0.5774 - acc: 0.80 - ETA: 40s - loss: 0.5782 - acc: 0.79 - ETA: 40s - loss: 0.5781 - acc: 0.79 - ETA: 40s - loss: 0.5784 - acc: 0.79 - ETA: 40s - loss: 0.5787 - acc: 0.79 - ETA: 39

In [46]:
def cifar10_predict(model, image_file):
  labels = ["airplane", "automobile", "bird", "cat",
    "deer", "dog", "frog", "horse", "ship", "truck"]

  img = keras.preprocessing.image.load_img(image_file, target_size=(32, 32))
  img_arr = np.expand_dims(img, axis=0) / 255.0
  result = model.predict_classes(img_arr)
  print("Result: {}".format(result))

In [36]:

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 6, 6, 64)         

In [47]:
cifar10_predict(model, "ceras1.jpg")

Result: [3]
